In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import pickle#쿠키값이나 여러 정보값 가져올 수 있게 하는 라이브러리
import time

options = Options()
options.add_argument("--start-maximized")
options.add_argument("window-size=1920x1080")
options.add_argument("lang=ko_KR")

service = Service(ChromeDriverManager().install())

driver = webdriver.Chrome(service=service, options=options)

url = "https://www.instagram.com/accounts/login"
driver.get(url)

print("수동으로 인증 시작!")
time.sleep(30)

#cookie 값을 저장하고 저장한 이름으로 오픈해서 실제 값을 넣어줌
with open("instagram_cookies.pkl", "wb") as f :#앞에 선언된 이름으로 실행해라
    pickle.dump(driver.get_cookies(), f)#쿠키 값 받아오기와서 윗줄로 보냄 dump => 받아온 쿠키값 정렬

print("쿠키 저장 완료")
driver.quit()

수동으로 인증 시작!
쿠키 저장 완료


In [9]:
# -*- coding: utf-8 -*-
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException, WebDriverException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pickle, time, random, re, csv
from urllib.parse import quote
# ==================== 설정 ====================
KEYWORD           = "여성영양제"
TARGET_CAPTIONS   = 350         # 실전 수집량
MAX_SCROLLS       = 400
VISIT_WAIT        = (0.5, 0.9)
SCROLL_SHORT      = (0.08, 0.18)
SCROLL_PAUSE      = (0.25, 0.55)
CSV_PATH          = "instagram_captions.csv"
INSTA_COOKIE_PKL  = "instagram_cookies.pkl"
FAST_HEADLESS     = False
BLOCK_MEDIA       = True
# =================================================
def make_driver():
    options = Options()
    options.page_load_strategy = "eager"
    options.add_argument("--start-maximized")
    options.add_argument("window-size=1920x1080")
    options.add_argument("lang=ko_KR")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--disable-infobars")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    if FAST_HEADLESS:
        options.add_argument("--headless=new")
    if BLOCK_MEDIA:
        prefs = {
            "profile.managed_default_content_settings.images": 2,
            "profile.managed_default_content_settings.fonts": 2,
            "profile.managed_default_content_settings.video": 2,
            "profile.managed_default_content_settings.sound": 2,
            "intl.accept_languages": "ko_KR",
        }
        options.add_experimental_option("prefs", prefs)
        options.add_argument("--blink-settings=imagesEnabled=false")
    service = Service(ChromeDriverManager().install())
    drv = webdriver.Chrome(service=service, options=options)
    drv.set_page_load_timeout(50)
    return drv
driver = make_driver()
wait = WebDriverWait(driver, 10)
actions = ActionChains(driver)
# ---------------- 로그인/쿠키 ----------------
driver.get("https://www.instagram.com/")
time.sleep(1.0)
try:
    with open(INSTA_COOKIE_PKL, "rb") as f:
        cookies = pickle.load(f)
    for c in cookies:
        if "expiry" in c and isinstance(c["expiry"], float):
            c["expiry"] = int(c["expiry"])
        try:
            driver.add_cookie(c)
        except Exception:
            pass
    driver.refresh()
    time.sleep(1.0)
except FileNotFoundError:
    print("[경고] instagram_cookies.pkl 없음 → 비로그인일 수 있음")
for txt in ["필수 쿠키만 허용", "쿠키 허용", "Only allow essential cookies", "Allow all cookies",
            "나중에 하기", "Not now", "지금 아니오", "나중에"]:
    try:
        btn = driver.find_element(By.XPATH, f"//button[normalize-space()='{txt}']")
        driver.execute_script("arguments[0].click();", btn)
        time.sleep(0.2)
    except:
        pass
# ---------------- 태그 페이지 ----------------
def goto_hashtag(tag):
    enc = quote(tag)
    for url in [f"https://www.instagram.com/explore/tags/{enc}/",
                f"https://www.instagram.com/explore/tags/{tag}/"]:
        driver.get(url)
        try:
            wait.until(EC.presence_of_element_located((By.TAG_NAME, "a")))
            return
        except TimeoutException:
            continue
goto_hashtag(KEYWORD)
def human_scroll_cycle():
    actions.send_keys(Keys.END).perform()
    time.sleep(random.uniform(*SCROLL_PAUSE))
    for _ in range(1):
        dy = random.randint(350, 900)
        driver.execute_script("window.scrollBy(0, arguments[0]);", dy)
        time.sleep(random.uniform(*SCROLL_SHORT))
# ---------------- JS 파싱(모달/페이지 공통) ----------------
JS_EXTRACT = r"""
function extractFrom(doc) {
  function txt(el){ return el ? (el.textContent || '').trim() : ''; }
  const dialog = doc.querySelector('div[role="dialog"]');
  const art = (dialog && dialog.querySelector('article')) || doc.querySelector('article');
  if(!art) return {caption:'', hashtags:[], postPath:'', canonical:'', og:''};
  // 더보기
  const more = Array.from(art.querySelectorAll('span,button')).find(e=>{
    const t=(e.textContent||'').trim();
    return ['더 보기','더보기','More','more','… 더보기','… more'].includes(t);
  });
  if(more){ more.click(); }
  // 첫 li 블록
  const li1 = art.querySelector('ul li:first-child');
  const capNode = (li1 && (li1.querySelector('h1[dir="auto"]') || li1.querySelector('span[dir="auto"]')))
                  || art.querySelector('h1[dir="auto"]');
  let caption = txt(capNode);
  if(!caption && li1) caption = txt(li1);
  // permalink 추정
  let postPath = '';
  const timeA = art.querySelector('a[href^="/p/"] time, a[href^="/reel/"] time');
  if(timeA && timeA.parentElement) postPath = timeA.parentElement.getAttribute('href') || '';
  const canonical = (doc.querySelector('link[rel="canonical"]')||{}).href || '';
  const og = (doc.querySelector('meta[property="og:url"]')||{}).content || '';
  // 해시태그
  const seen = new Set(); const tags = [];
  if(li1){
    li1.querySelectorAll('a[href^="/explore/tags/"]').forEach(a=>{
      let raw = (a.textContent||'').trim().replace(/^#\s*/,'')
                 .replace(/[\u200b-\u200f\u202a-\u202e\u2060\s]+/g,'');
      if(raw && !seen.has(raw)){ seen.add(raw); tags.push(raw); }
    });
  }
  if(!tags.length && caption){
    (caption.match(/#([0-9A-Za-z_가-힣_]+)/g)||[]).forEach(s=>{
      const t = s.slice(1);
      if(t && !seen.has(t)){ seen.add(t); tags.push(t); }
    });
  }
  return {caption, hashtags:tags, postPath, canonical, og};
}
return extractFrom(document);
"""
def extract_current():
    d = driver.execute_script(JS_EXTRACT)
    cap = (d.get("caption") or "").strip()
    tags = d.get("hashtags") or []
    url = ""
    if d.get("postPath") and str(d["postPath"]).startswith("/"):
        url = "https://www.instagram.com" + d["postPath"]
    elif d.get("canonical"):
        url = d["canonical"]
    elif d.get("og"):
        url = d["og"]
    else:
        url = driver.current_url
    return url, cap, tags
def modal_present():
    try:
        return driver.execute_script("return !!document.querySelector('div[role=\"dialog\"] article');")
    except:
        return False
def open_first_modal():
    anchors = driver.find_elements(By.CSS_SELECTOR, "a[href*='/p/'], a[href*='/reel/']")
    for a in anchors:
        try:
            driver.execute_script("arguments[0].click();", a)
            time.sleep(random.uniform(*VISIT_WAIT))
            if modal_present():
                return True
        except:
            pass
    return False
# --- 핵심: “다음 포스트”만 클릭(캐러셀 제외) ---
def click_next_post():
    """
    모달 바깥(ancestor::article 아닌) '다음/Next' 컨트롤만 클릭.
    실패 시 JS로 후보를 찾아 클릭하고, 그래도 안되면 ArrowRight 키 사용.
    """
    # 1) XPath: article 바깥
    xps = [
        "//div[@role='dialog']//*[self::button or self::div[@role='button'] or self::a]"
        "[@aria-label='다음' or @aria-label='Next'][not(ancestor::article)]",
        "//div[@role='dialog']//*[name()='svg' and (@aria-label='다음' or @aria-label='Next')]"
        "/ancestor::*[self::button or self::div[@role='button'] or self::a][1][not(ancestor::article)]",
    ]
    for xp in xps:
        try:
            btn = driver.find_element(By.XPATH, xp)
            driver.execute_script("arguments[0].click();", btn)
            time.sleep(random.uniform(*VISIT_WAIT))
            return True
        except:
            continue
    # 2) JS로 후보 찾아 클릭(역시 article 바깥만)
    js = r"""
    const d=document.querySelector('div[role="dialog"]'); if(!d) return false;
    const art=d.querySelector('article');
    const cands=[...d.querySelectorAll('[aria-label="다음"],[aria-label="Next"]')];
    const target=cands.find(el=>!art || !el.closest('article'));
    if(target){ target.click(); return true; }
    return false;
    """
    try:
        ok = driver.execute_script(js)
        if ok:
            time.sleep(random.uniform(*VISIT_WAIT))
            return True
    except:
        pass
    # 3) 최후: ArrowRight (모달 컨테이너에 포커스)
    try:
        dlg = driver.find_element(By.CSS_SELECTOR, "div[role='dialog']")
        driver.execute_script("arguments[0].focus();", dlg)
        dlg.click()
        actions.send_keys(Keys.ARROW_RIGHT).perform()
        time.sleep(random.uniform(*VISIT_WAIT))
        return True
    except:
        return False
def close_modal_if_open():
    if not modal_present(): return
    xps = [
        "//div[@role='dialog']//button[@aria-label='닫기' or @aria-label='Close']",
        "//div[@role='dialog']//div[@role='button' and (@aria-label='닫기' or @aria-label='Close')]",
    ]
    for xp in xps:
        try:
            btn = driver.find_element(By.XPATH, xp)
            driver.execute_script("arguments[0].click();", btn)
            time.sleep(0.2)
            return
        except:
            pass
    try:
        actions.send_keys(Keys.ESCAPE).perform()
        time.sleep(0.2)
    except:
        pass
# ---------------- 빠른 순회 로직 ----------------
results, visited = [], set()

# 크롤링 회차 카운터 (추가)
crawl_iter = 0

# 1) 모달 열기(없으면 스크롤하며 시도)
if not open_first_modal():
    tries = 0
    while tries < 8 and not modal_present():
        human_scroll_cycle()
        if open_first_modal(): break
        tries += 1
# 2) 모달에서 다음 "포스트"로만 순회
same_guard = 0
while modal_present() and len(results) < TARGET_CAPTIONS:
    try:
        crawl_iter += 1  # 몇 번째 크롤링인지 표시용 (추가)
        print(f"[크롤링 #{crawl_iter}] 모달 뷰에서 포스트 처리 중...")
        url, cap, tags = extract_current()
        if url not in visited and (cap or tags):
            visited.add(url)
            results.append({"url": url, "caption": cap, "hashtags": tags})
            print(f"  ↳ 수집 완료 ({len(results)}/{TARGET_CAPTIONS}): {url}")
        prev_url = url
        if not click_next_post():
            close_modal_if_open()
            human_scroll_cycle()
            if not open_first_modal():
                human_scroll_cycle()
                if not open_first_modal():
                    break
            same_guard = 0
            continue
        # “다음” 눌렀는데 같은 포스트라면(캐러셀만 넘어간 경우) → 다시 시도
        url2, _, _ = extract_current()
        if url2 == prev_url:
            same_guard += 1
            if same_guard >= 2:
                close_modal_if_open()
                human_scroll_cycle()
                open_first_modal()
                same_guard = 0
            continue
        same_guard = 0
    except WebDriverException:
        close_modal_if_open()
        human_scroll_cycle()
        open_first_modal()
# 3) 백업: 모달이 막히면 링크 방문
if len(results) < TARGET_CAPTIONS:
    close_modal_if_open()
    have = set()
    for _ in range(20):
        anchors = driver.find_elements(By.CSS_SELECTOR, "a[href*='/p/'], a[href*='/reel/']")
        for a in anchors:
            href = a.get_attribute("href")
            if href and ("/p/" in href or "/reel/" in href):
                have.add(href)
        if len(have) >= TARGET_CAPTIONS*1.3: break
        human_scroll_cycle()
    links = [h for h in have if h not in visited]
    for url in links:
        if len(results) >= TARGET_CAPTIONS: break
        try:
            crawl_iter += 1  # 몇 번째 크롤링인지 표시용 (추가)
            print(f"[크롤링 #{crawl_iter}] 백업 모드로 개별 URL 방문: {url}")
            driver.get(url)
            time.sleep(random.uniform(*VISIT_WAIT))
            u, cap, tags = extract_current()
            if u not in visited and (cap or tags):
                visited.add(u)
                results.append({"url": u, "caption": cap, "hashtags": tags})
                print(f"  ↳ 수집 완료 ({len(results)}/{TARGET_CAPTIONS}): {u}")
        except Exception:
            continue
# ---------------- CSV 저장 ----------------
with open(CSV_PATH, "w", newline="", encoding="utf-8-sig") as f:
    w = csv.writer(f)
    w.writerow(["url", "caption", "hashtags"])
    for r in results:
        w.writerow([r["url"], r["caption"], ";".join(r["hashtags"])])
print(f"[완료] 캡션 {len(results)}건 저장 → {CSV_PATH}")
driver.quit()


[크롤링 #1] 모달 뷰에서 포스트 처리 중...
  ↳ 수집 완료 (1/350): https://www.instagram.com/p/C6IozV_Stqw/c/18075538321435958/
[크롤링 #2] 모달 뷰에서 포스트 처리 중...
  ↳ 수집 완료 (2/350): https://www.instagram.com/p/C4zauEOxhMP/
[크롤링 #3] 모달 뷰에서 포스트 처리 중...
  ↳ 수집 완료 (3/350): https://www.instagram.com/p/DN0YxJy5PBU/c/18043207682660395/
[크롤링 #4] 모달 뷰에서 포스트 처리 중...
  ↳ 수집 완료 (4/350): https://www.instagram.com/p/DQIg89UEzCy/
[크롤링 #5] 모달 뷰에서 포스트 처리 중...
  ↳ 수집 완료 (5/350): https://www.instagram.com/p/DDtTamgpWM-/c/18082354981989659/
[크롤링 #6] 모달 뷰에서 포스트 처리 중...
  ↳ 수집 완료 (6/350): https://www.instagram.com/p/DOqldSpkiks/
[크롤링 #7] 모달 뷰에서 포스트 처리 중...
  ↳ 수집 완료 (7/350): https://www.instagram.com/p/DNiV6crJVtP/c/18051228275233046/
[크롤링 #8] 모달 뷰에서 포스트 처리 중...
  ↳ 수집 완료 (8/350): https://www.instagram.com/p/DJjWsuKvUVr/
[크롤링 #9] 모달 뷰에서 포스트 처리 중...
  ↳ 수집 완료 (9/350): https://www.instagram.com/p/DFRyDO0vWwn/
[크롤링 #10] 모달 뷰에서 포스트 처리 중...
  ↳ 수집 완료 (10/350): https://www.instagram.com/p/DIJEKI9zMz_/c/18500221699023282/
[크롤링 #11] 모달 뷰에서 포

InvalidSessionIdException: Message: invalid session id; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#invalidsessionidexception
Stacktrace:
	GetHandleVerifier [0x0x72fe43+66515]
	GetHandleVerifier [0x0x72fe84+66580]
	(No symbol) [0x0x51da6b]
	(No symbol) [0x0x55b5ab]
	(No symbol) [0x0x58b086]
	(No symbol) [0x0x58667c]
	(No symbol) [0x0x585bf3]
	(No symbol) [0x0x4ee72d]
	(No symbol) [0x0x4eecae]
	(No symbol) [0x0x4ef14d]
	GetHandleVerifier [0x0x987353+2521315]
	GetHandleVerifier [0x0x9822d3+2500707]
	GetHandleVerifier [0x0x757c94+229924]
	GetHandleVerifier [0x0x7481f8+165768]
	GetHandleVerifier [0x0x74ecad+193085]
	(No symbol) [0x0x4ee3e3]
	(No symbol) [0x0x4edb60]
	GetHandleVerifier [0x0xacc20f+3852191]
	BaseThreadInitThunk [0x0x767c5d49+25]
	RtlInitializeExceptionChain [0x0x77add6db+107]
	RtlGetAppContainerNamedObjectPath [0x0x77add661+561]


In [16]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException, WebDriverException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import pickle, time, random, re, csv
from urllib.parse import quote

# ==================== 설정 ====================
KEYWORD           = "이너뷰티"
TARGET_CAPTIONS   = 1000       # 수집 목표
MAX_SCROLLS       = 1100        # 태그 피드 스크롤 상한
VISIT_WAIT        = (0.55, 1.1)
SCROLL_SHORT      = (0.08, 0.18)
SCROLL_PAUSE      = (0.25, 0.55)
CSV_PATH          = "instagram_captions.csv"
INSTA_COOKIE_PKL  = "instagram_cookies.pkl"
FAST_HEADLESS     = False
BLOCK_MEDIA       = True       # 텍스트만 크롤링 → 미디어 차단으로 속도 ↑
# =================================================

def make_driver():
    options = Options()
    options.page_load_strategy = "eager"
    options.add_argument("--start-maximized")
    options.add_argument("window-size=1920x1080")
    options.add_argument("lang=ko_KR")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--disable-infobars")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    if FAST_HEADLESS:
        options.add_argument("--headless=new")
    if BLOCK_MEDIA:
        prefs = {
            "profile.managed_default_content_settings.images": 2,
            "profile.managed_default_content_settings.fonts": 2,
            "profile.managed_default_content_settings.video": 2,
            "profile.managed_default_content_settings.sound": 2,
            "intl.accept_languages": "ko_KR",
        }
        options.add_experimental_option("prefs", prefs)
        options.add_argument("--blink-settings=imagesEnabled=false")
    service = Service(ChromeDriverManager().install())
    drv = webdriver.Chrome(service=service, options=options)
    drv.set_page_load_timeout(50)
    return drv

driver = make_driver()
wait = WebDriverWait(driver, 10)
actions = ActionChains(driver)

# ---------------- 로그인/쿠키 ----------------
driver.get("https://www.instagram.com/")
time.sleep(1.0)
try:
    with open(INSTA_COOKIE_PKL, "rb") as f:
        cookies = pickle.load(f)
    for c in cookies:
        if "expiry" in c and isinstance(c["expiry"], float):
            c["expiry"] = int(c["expiry"])
        try:
            driver.add_cookie(c)
        except Exception:
            pass
    driver.refresh()
    time.sleep(1.0)
except FileNotFoundError:
    print("[경고] instagram_cookies.pkl 없음 → 비로그인일 수 있음")

for txt in ["필수 쿠키만 허용", "쿠키 허용", "Only allow essential cookies", "Allow all cookies",
            "나중에 하기", "Not now", "지금 아니오", "나중에"]:
    try:
        btn = driver.find_element(By.XPATH, f"//button[normalize-space()='{txt}']")
        driver.execute_script("arguments[0].click();", btn)
        time.sleep(0.2)
    except:
        pass

# ---------------- 태그 페이지 ----------------
def goto_hashtag(tag):
    enc = quote(tag)
    for url in [f"https://www.instagram.com/explore/tags/{enc}/",
                f"https://www.instagram.com/explore/tags/{tag}/"]:
        driver.get(url)
        try:
            wait.until(EC.presence_of_element_located((By.TAG_NAME, "a")))
            return
        except TimeoutException:
            continue

goto_hashtag(KEYWORD)

def human_scroll_cycle():
    actions.send_keys(Keys.END).perform()
    time.sleep(random.uniform(*SCROLL_PAUSE))
    for _ in range(1):
        dy = random.randint(350, 900)
        driver.execute_script("window.scrollBy(0, arguments[0]);", dy)
        time.sleep(random.uniform(*SCROLL_SHORT))

# ---------------- JS 파싱(모달/페이지 공통) ----------------
JS_EXTRACT = r"""
function extractFrom(doc) {
  function txt(el){ return el ? (el.textContent || '').trim() : ''; }
  const dialog = doc.querySelector('div[role="dialog"]');
  const art = (dialog && dialog.querySelector('article')) || doc.querySelector('article');
  if(!art) return {caption:'', hashtags:[], postPath:'', canonical:'', og:'', publishedISO:'', publishedText:''};

  // 더보기
  const more = Array.from(art.querySelectorAll('span,button')).find(e=>{
    const t=(e.textContent||'').trim();
    return ['더 보기','더보기','More','more','… 더보기','… more'].includes(t);
  });
  if(more){ more.click(); }

  // 첫 li 블록
  const li1 = art.querySelector('ul li:first-child');
  const capNode = (li1 && (li1.querySelector('h1[dir="auto"]') || li1.querySelector('span[dir="auto"]')))
                  || art.querySelector('h1[dir="auto"]');
  let caption = txt(capNode);
  if(!caption && li1) caption = txt(li1);

  // permalink & 시간
  let postPath = '';
  let t = art.querySelector('a[href^="/p/"] time, a[href^="/reel/"] time') || art.querySelector('time');
  if(t && t.parentElement && t.parentElement.getAttribute('href')) {
    postPath = t.parentElement.getAttribute('href') || '';
  }
  const publishedISO  = (t && t.getAttribute('datetime')) || (doc.querySelector('meta[property="article:published_time"]')||{}).content || '';
  const publishedText = (t && (t.getAttribute('title') || t.textContent || '').trim()) || '';

  const canonical = (doc.querySelector('link[rel="canonical"]')||{}).href || '';
  const og = (doc.querySelector('meta[property="og:url"]')||{}).content || '';

  // 해시태그
  const seen = new Set(); const tags = [];
  if(li1){
    li1.querySelectorAll('a[href^="/explore/tags/"]').forEach(a=>{
      let raw = (a.textContent||'').trim().replace(/^#\s*/,'')
                 .replace(/[\u200b-\u200f\u202a-\u202e\u2060\s]+/g,'');
      if(raw && !seen.has(raw)){ seen.add(raw); tags.push(raw); }
    });
  }
  if(!tags.length && caption){
    (caption.match(/#([0-9A-Za-z_가-힣_]+)/g)||[]).forEach(s=>{
      const tt = s.slice(1);
      if(tt && !seen.has(tt)){ seen.add(tt); tags.push(tt); }
    });
  }
  return {caption, hashtags:tags, postPath, canonical, og, publishedISO, publishedText};
}
return extractFrom(document);
"""

def extract_current():
    """현재 모달/페이지에서 url, caption, tags, 날짜 추출"""
    d = driver.execute_script(JS_EXTRACT)
    cap = (d.get("caption") or "").strip()
    tags = d.get("hashtags") or []
    url = ""
    if d.get("postPath") and str(d["postPath"]).startswith("/"):
        url = "https://www.instagram.com" + d["postPath"]
    elif d.get("canonical"):
        url = d["canonical"]
    elif d.get("og"):
        url = d["og"]
    else:
        url = driver.current_url
    pub_iso  = (d.get("publishedISO") or "").strip()
    pub_text = (d.get("publishedText") or "").strip()
    return url, cap, tags, pub_iso, pub_text

def modal_present():
    try:
        return driver.execute_script("return !!document.querySelector('div[role=\"dialog\"] article');")
    except:
        return False

def open_first_modal():
    """피드의 첫 썸네일을 클릭해서 모달오픈"""
    anchors = driver.find_elements(By.CSS_SELECTOR, "a[href*='/p/'], a[href*='/reel/']")
    for a in anchors:
        try:
            driver.execute_script("arguments[0].click();", a)
            time.sleep(random.uniform(*VISIT_WAIT))
            if modal_present():
                return True
        except:
            pass
    return False

# --- 핵심: "다음 포스트" 전용 클릭 ---
def click_next_post_and_wait(prev_key, timeout=6):
    """
    1) article 바깥 '다음/Next'만 클릭
    2) 클릭 후 prev_key와 다른 포스트가 뜰 때까지 대기
    """
    clicked = False
    try:
        js = r"""
        const d=document.querySelector('div[role="dialog"]'); if(!d) return false;
        const art=d.querySelector('article');
        const cands=[...d.querySelectorAll('[aria-label="다음"],[aria-label="Next"]')];
        const t=cands.find(el=>!art || !el.closest('article'));
        if(t){ t.click(); return true; }
        return false;
        """
        clicked = driver.execute_script(js)
    except Exception:
        clicked = False

    if not clicked:
        try:
            dlg = driver.find_element(By.CSS_SELECTOR, "div[role='dialog']")
            driver.execute_script("arguments[0].focus();", dlg)
            dlg.click()
            actions.send_keys(Keys.ARROW_RIGHT).perform()
            clicked = True
        except:
            clicked = False

    if not clicked:
        return False

    end = time.time() + timeout
    while time.time() < end:
        try:
            url2, _, _, _, _ = extract_current()
            if url2 and url2 != prev_key:
                return True
        except:
            pass
        time.sleep(0.25)
    return False

def close_modal_if_open():
    if not modal_present(): return
    xps = [
        "//div[@role='dialog']//button[@aria-label='닫기' or @aria-label='Close']",
        "//div[@role='dialog']//div[@role='button' and (@aria-label='닫기' or @aria-label='Close')]",
    ]
    for xp in xps:
        try:
            btn = driver.find_element(By.XPATH, xp)
            driver.execute_script("arguments[0].click();", btn)
            time.sleep(0.2)
            return
        except:
            pass
    try:
        actions.send_keys(Keys.ESCAPE).perform()
        time.sleep(0.2)
    except:
        pass

# ---------------- 빠른 순회 로직 ----------------
results, visited = [], set()

# 1) 모달 열기(없으면 스크롤하며 시도)
if not open_first_modal():
    tries = 0
    while tries < 8 and not modal_present():
        human_scroll_cycle()
        if open_first_modal(): break
        tries += 1

# 2) 모달에서 "다음 포스트" 순회
stuck_guard = 0
hop_guard   = 0   # 안전 상한 (무한 루프 방지)

while modal_present() and len(results) < TARGET_CAPTIONS and hop_guard < TARGET_CAPTIONS * 3:
    hop_guard += 1
    try:
        url, cap, tags, pub_iso, pub_text = extract_current()
        if url not in visited and (cap or tags):
            visited.add(url)
            results.append({"url": url, "caption": cap, "hashtags": tags,
                            "date_iso": pub_iso, "date_text": pub_text})
        prev_key = url

        moved = click_next_post_and_wait(prev_key, timeout=6)
        if not moved:
            close_modal_if_open()
            human_scroll_cycle()
            if not open_first_modal():
                human_scroll_cycle()
                if not open_first_modal():
                    break
            stuck_guard = 0
            continue

        url2, _, _, _, _ = extract_current()
        if url2 == prev_key:
            stuck_guard += 1
            if stuck_guard >= 2:
                close_modal_if_open()
                human_scroll_cycle()
                open_first_modal()
                stuck_guard = 0
            continue
        stuck_guard = 0

    except WebDriverException:
        close_modal_if_open()
        human_scroll_cycle()
        open_first_modal()

# 3) 백업: 모달이 막히면 개별 링크 방문
if len(results) < TARGET_CAPTIONS:
    close_modal_if_open()
    have = set()
    feed_scrolls = 0
    while len(have) < TARGET_CAPTIONS*1.3 and feed_scrolls < MAX_SCROLLS:
        anchors = driver.find_elements(By.CSS_SELECTOR, "a[href*='/p/'], a[href*='/reel/']")
        for a in anchors:
            href = a.get_attribute("href")
            if href and ("/p/" in href or "/reel/" in href):
                have.add(href)
        human_scroll_cycle()
        feed_scrolls += 1
    links = [h for h in have if h not in visited]
    for url in links:
        if len(results) >= TARGET_CAPTIONS: break
        try:
            driver.get(url)
            time.sleep(random.uniform(*VISIT_WAIT))
            u, cap, tags, pub_iso, pub_text = extract_current()
            if u not in visited and (cap or tags):
                visited.add(u)
                results.append({"url": u, "caption": cap, "hashtags": tags,
                                "date_iso": pub_iso, "date_text": pub_text})
        except Exception:
            continue

# ---------------- CSV 저장 ----------------
with open(CSV_PATH, "w", newline="", encoding="utf-8-sig") as f:
    w = csv.writer(f)
    # 날짜 2개 컬럼만 **추가** (기존 컬럼 보존)
    w.writerow(["url", "caption", "hashtags", "date_iso", "date_text"])
    for r in results:
        w.writerow([r["url"], r["caption"], ";".join(r["hashtags"]), r.get("date_iso",""), r.get("date_text","")])

print(f"[완료] 캡션 {len(results)}건 저장 → {CSV_PATH}")
driver.quit()

MaxRetryError: HTTPConnectionPool(host='localhost', port=56146): Max retries exceeded with url: /session/eabaaa0fffd8dbe70b69e2a3367ce10f/actions (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000002B1339485A0>: Failed to establish a new connection: [WinError 10061] 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다'))

In [15]:
# -*- coding: utf-8 -*-
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException, WebDriverException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pickle, time, random, re, csv
from urllib.parse import quote

# ==================== 설정 (안정성 강화) ====================
KEYWORD           = "이너뷰티"
TARGET_CAPTIONS   = 300             # 실전 수집량
MAX_SCROLLS       = 400
VISIT_WAIT        = (2.0, 4.0)      # 개별 방문 및 모달 로드 후 대기 시간 대폭 증가 (차단 회피)
SCROLL_SHORT      = (0.5, 1.0)      # 스크롤 동작 간격 증가 (사람 흉내)
SCROLL_PAUSE      = (1.5, 3.0)      # 스크롤 후 대기 시간 증가 (사람 흉내)
CSV_PATH          = "instagram_captions_with_time.csv" # CSV 파일명 변경
INSTA_COOKIE_PKL  = "instagram_cookies.pkl"
FAST_HEADLESS     = False
BLOCK_MEDIA       = True
NEW_TIMEOUT       = 70              # 페이지 로드 타임아웃 50초에서 70초로 증가
# =================================================

def make_driver():
    options = Options()
    # 페이지 로딩 전략을 'eager'에서 'normal'로 변경하여 모든 리소스 로드 대기
    options.page_load_strategy = "normal"
    options.add_argument("--start-maximized")
    options.add_argument("window-size=1920x1080")
    options.add_argument("lang=ko_KR")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--disable-infobars")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    if FAST_HEADLESS:
        options.add_argument("--headless=new")
    if BLOCK_MEDIA:
        prefs = {
            "profile.managed_default_content_settings.images": 2,
            "profile.managed_default_content_settings.fonts": 2,
            "profile.managed_default_content_settings.video": 2,
            "profile.managed_default_content_settings.sound": 2,
            "intl.accept_languages": "ko_KR",
        }
        options.add_experimental_option("prefs", prefs)
        options.add_argument("--blink-settings=imagesEnabled=false")
    service = Service(ChromeDriverManager().install())
    drv = webdriver.Chrome(service=service, options=options)
    drv.set_page_load_timeout(NEW_TIMEOUT)
    return drv

driver = make_driver()
wait = WebDriverWait(driver, 15) # WebDriverWait 시간도 10초에서 15초로 증가
actions = ActionChains(driver)

# ---------------- 로그인/쿠키 ----------------
driver.get("https://www.instagram.com/")
time.sleep(1.0)
try:
    with open(INSTA_COOKIE_PKL, "rb") as f:
        cookies = pickle.load(f)
    for c in cookies:
        if "expiry" in c and isinstance(c["expiry"], float):
            c["expiry"] = int(c["expiry"])
        try:
            driver.add_cookie(c)
        except Exception:
            pass
    driver.refresh()
    time.sleep(1.0)
except FileNotFoundError:
    print("[경고] instagram_cookies.pkl 없음 → 비로그인일 수 있음")

for txt in ["필수 쿠키만 허용", "쿠키 허용", "Only allow essential cookies", "Allow all cookies",
             "나중에 하기", "Not now", "지금 아니오", "나중에"]:
    try:
        btn = driver.find_element(By.XPATH, f"//button[normalize-space()='{txt}']")
        driver.execute_script("arguments[0].click();", btn)
        time.sleep(0.2)
    except:
        pass

# ---------------- 태그 페이지 ----------------
def goto_hashtag(tag):
    enc = quote(tag)
    for url in [f"https://www.instagram.com/explore/tags/{enc}/",
                f"https://www.instagram.com/explore/tags/{tag}/"]:
        driver.get(url)
        try:
            wait.until(EC.presence_of_element_located((By.TAG_NAME, "a")))
            return
        except TimeoutException:
            continue

goto_hashtag(KEYWORD)

def human_scroll_cycle():
    # 설정값 (SCROLL_PAUSE, SCROLL_SHORT)을 사용해 느린 스크롤 구현
    actions.send_keys(Keys.END).perform()
    time.sleep(random.uniform(*SCROLL_PAUSE))
    dy = random.randint(350, 900)
    driver.execute_script("window.scrollBy(0, arguments[0]);", dy)
    time.sleep(random.uniform(*SCROLL_SHORT))

# ---------------- JS 파싱 (게시 시간 + 퍼머링크 추출 보강) ----------------
JS_EXTRACT = r"""
function extractFrom(doc) {
  function txt(el){ return el ? (el.textContent || '').trim() : ''; }
  const dialog = doc.querySelector('div[role="dialog"]');
  const art = (dialog && dialog.querySelector('article')) || doc.querySelector('article');

  let postingTime = '';
  if(!art) return {caption:'', hashtags:[], postPath:'', canonical:'', og:'', postingTime: postingTime};

  // ===== 게시 시간 추출 =====
  const timeEl = art.querySelector('time[datetime]');
  if (timeEl) {
    postingTime = timeEl.getAttribute('datetime') || '';
  } else {
    const timeA = art.querySelector('a[href^="/p/"] time, a[href^="/reel/"] time');
    if (timeA) postingTime = txt(timeA);
  }

  // ===== 더보기 클릭 =====
  const more = Array.from(art.querySelectorAll('span,button')).find(e=>{
    const t=(e.textContent||'').trim();
    return ['더 보기','더보기','More','more','… 더보기','… more'].includes(t);
  });
  if(more){ more.click(); }

  // ===== 캡션 =====
  const li1 = art.querySelector('ul li:first-child');
  const capNode = (li1 && (li1.querySelector('h1[dir="auto"]') || li1.querySelector('span[dir="auto"]')))
                  || art.querySelector('h1[dir="auto"]');
  let caption = txt(capNode);
  if(!caption && li1) caption = txt(li1);

  // ===== permalink 추정 (보강) =====
  let postPath = '';
  // 1) 모달 article 내에서 포스트 앵커 자체를 우선적으로 찾기
  const linkCand = art.querySelector('a[href^="/p/"], a[href^="/reel/"]');
  if (linkCand) {
    postPath = linkCand.getAttribute('href') || '';
  }
  // 2) 백업: time이 앵커 안에 있을 때 (기존 방식)
  if (!postPath) {
    const timeA = art.querySelector('a[href^="/p/"] time, a[href^="/reel/"] time');
    if (timeA && timeA.parentElement) postPath = timeA.parentElement.getAttribute('href') || '';
  }
  // 3) 최종 백업: canonical / og
  const canonical = (doc.querySelector('link[rel="canonical"]')||{}).href || '';
  const og = (doc.querySelector('meta[property="og:url"]')||{}).content || '';

  // ===== 해시태그 =====
  const seen = new Set(); const tags = [];
  if(li1){
    li1.querySelectorAll('a[href^="/explore/tags/"]').forEach(a=>{
      let raw = (a.textContent||'').trim().replace(/^#\s*/,'')
                  .replace(/[\u200b-\u200f\u202a-\u202e\u2060\s]+/g,'');
      if(raw && !seen.has(raw)){ seen.add(raw); tags.push(raw); }
    });
  }
  if(!tags.length && caption){
    (caption.match(/#([0-9A-Za-z_가-힣_]+)/g)||[]).forEach(s=>{
      const t = s.slice(1);
      if(t && !seen.has(t)){ seen.add(t); tags.push(t); }
    });
  }

  return {caption, hashtags:tags, postPath, canonical, og, postingTime};
}
return extractFrom(document);
"""

def extract_current():
    d = driver.execute_script(JS_EXTRACT)
    cap = (d.get("caption") or "").strip()
    tags = d.get("hashtags") or []
    post_time = (d.get("postingTime") or "").strip() # 게시 시간 추출 추가
    url = ""
    if d.get("postPath") and str(d["postPath"]).startswith("/"):
        url = "https://www.instagram.com" + d["postPath"]
    elif d.get("canonical"):
        url = d["canonical"]
    elif d.get("og"):
        url = d["og"]
    else:
        url = driver.current_url
    return url, cap, tags, post_time # 게시 시간 반환 추가

def modal_present():
    try:
        # 모달이 열렸는지 확인
        return driver.execute_script("return !!document.querySelector('div[role=\"dialog\"] article');")
    except:
        return False

# 시도한 썸네일 재클릭 방지 세트
tried_thumbs = set()

# open_first_modal: 같은 썸네일 반복 클릭 방지 + 뷰포트 중앙 스크롤
def open_first_modal():
    anchors = driver.find_elements(By.CSS_SELECTOR, "a[href*='/p/'], a[href*='/reel/']")
    for a in anchors:
        try:
            href = a.get_attribute("href") or ""
            if href in tried_thumbs:
                continue
            tried_thumbs.add(href)

            # 클릭 전 뷰포트 중앙 이동 (안정성 증대)
            driver.execute_script("arguments[0].scrollIntoView({block:'center'});", a)
            time.sleep(random.uniform(0.3, 0.7))
            driver.execute_script("arguments[0].click();", a)

            # 모달의 article이 나타날 때까지 명시적으로 대기
            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[role="dialog"] article')))
            time.sleep(random.uniform(*VISIT_WAIT))
            return True
        except TimeoutException:
            close_modal_if_open()
            continue
        except WebDriverException:
            continue
    return False

# --- 핵심: “다음 포스트”만 클릭(캐러셀 제외) + 실제 변경 검증 ---
def click_next_post():
    """
    모달 바깥(ancestor::article 아닌) '다음/Next' 컨트롤만 클릭.
    실패 시 JS로 후보를 찾아 클릭하고, 그래도 안되면 ArrowRight 키 사용.
    이동 후 실제 콘텐츠가 바뀌었는지도 검증.
    """
    # 0) 이동 전 기준 스냅샷(변경 검증용)
    try:
        u0, c0, *_ = extract_current()
    except:
        u0, c0 = "", ""

    # 1) XPath: article 바깥
    xps = [
        "//div[@role='dialog']//*[self::button or self::div[@role='button'] or self::a]"
        "[@aria-label='다음' or @aria-label='Next'][not(ancestor::article)]",
        "//div[@role='dialog']//*[name()='svg' and (@aria-label='다음' or @aria-label='Next')]"
        "/ancestor::*[self::button or self::div[@role='button'] or self::a][1][not(ancestor::article)]",
    ]
    for xp in xps:
        try:
            btn = driver.find_element(By.XPATH, xp)
            driver.execute_script("arguments[0].click();", btn)
            time.sleep(random.uniform(*VISIT_WAIT))
            u1, c1, *_ = extract_current()
            if u1 != u0 or c1 != c0:
                return True
        except:
            continue

    # 2) JS로 후보 찾아 클릭(역시 article 바깥만)
    js = r"""
    const d=document.querySelector('div[role="dialog"]'); if(!d) return false;
    const art=d.querySelector('article');
    const cands=[...d.querySelectorAll('[aria-label="다음"],[aria-label="Next"]')];
    const target=cands.find(el=>!art || !el.closest('article'));
    if(target){ target.click(); return true; }
    return false;
    """
    try:
        ok = driver.execute_script(js)
        if ok:
            time.sleep(random.uniform(*VISIT_WAIT))
            u2, c2, *_ = extract_current()
            if u2 != u0 or c2 != c0:
                return True
    except:
        pass

    # 3) 최후: ArrowRight 두 번 시도 + 변경 검증
    try:
        dlg = driver.find_element(By.CSS_SELECTOR, "div[role='dialog']")
        driver.execute_script("arguments[0].focus();", dlg)
        dlg.click()

        actions.send_keys(Keys.ARROW_RIGHT).perform()
        time.sleep(random.uniform(*VISIT_WAIT))
        u3, c3, *_ = extract_current()
        if u3 != u0 or c3 != c0:
            return True

        actions.send_keys(Keys.ARROW_RIGHT).perform()
        time.sleep(random.uniform(*VISIT_WAIT))
        u4, c4, *_ = extract_current()
        return (u4 != u0 or c4 != c0)
    except:
        return False

def close_modal_if_open():
    if not modal_present(): return
    xps = [
        "//div[@role='dialog']//button[@aria-label='닫기' or @aria-label='Close']",
        "//div[@role='dialog']//div[@role='button' and (@aria-label='닫기' or @aria-label='Close')]",
    ]
    for xp in xps:
        try:
            btn = driver.find_element(By.XPATH, xp)
            driver.execute_script("arguments[0].click();", btn)
            time.sleep(0.5)
            return
        except:
            pass
    try:
        actions.send_keys(Keys.ESCAPE).perform()
        time.sleep(0.5)
    except:
        pass

# ---------------- 빠른 순회 로직 (게시 시간 저장 + 무진행 안전판) ----------------
results, visited = [], set()
crawl_iter = 0
no_progress = 0  # 결과 증가 없는 회수 카운터

# 1) 모달 열기 시도 (스크롤 시도 횟수 15회로 증가)
if not open_first_modal():
    tries = 0
    while tries < 15 and not modal_present():
        human_scroll_cycle()
        if open_first_modal(): break
        tries += 1

# 2) 모달에서 다음 "포스트"로만 순회
same_guard = 0
while modal_present() and len(results) < TARGET_CAPTIONS:
    try:
        crawl_iter += 1
        print(f"[크롤링 #{crawl_iter}] 모달 뷰에서 포스트 처리 중...")

        # article 요소가 완전히 로드될 때까지 다시 대기 (추가 안전 장치)
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'article')))
        time.sleep(random.uniform(1.0, 2.0))

        prev_len = len(results)

        url, cap, tags, post_time = extract_current() # 게시 시간 추출
        if url not in visited and (cap or tags):
            visited.add(url)
            results.append({"url": url, "caption": cap, "hashtags": tags, "posting_time": post_time}) # 게시 시간 저장
            print(f"  ↳ 수집 완료 ({len(results)}/{TARGET_CAPTIONS}): {url} (시간: {post_time[:19]})")

        # 진행 상황 체크
        if len(results) == prev_len:
            no_progress += 1
        else:
            no_progress = 0

        prev_url = url

        if not click_next_post():
            # 다음 포스트 이동 실패 시 재시도 로직 강화
            close_modal_if_open()
            human_scroll_cycle()
            if not open_first_modal():
                human_scroll_cycle()
                if not open_first_modal():
                    break
            same_guard = 0
            continue

        # “다음” 눌렀는데 같은 포스트라면(캐러셀만 넘어간 경우) → 다시 시도
        url2, _, _, _ = extract_current() # 게시 시간 추출은 여기서는 무시
        if url2 == prev_url:
            same_guard += 1
            if same_guard >= 3:
                close_modal_if_open()
                human_scroll_cycle()
                open_first_modal()
                same_guard = 0
            continue
        same_guard = 0

        # 무진행 안전판: 일정 횟수 연속 진전 없으면 강제 범위 이동
        if no_progress >= 8:
            print("  ↳ 진전 없음 감지 → 모달 닫고 다른 범위로 이동")
            close_modal_if_open()
            for _ in range(4):
                human_scroll_cycle()
            open_first_modal()
            no_progress = 0
            continue

    except WebDriverException:
        # 예외 발생 시 모달 닫고, 스크롤 후 다시 모달 열기 재시도
        print(f"  ↳ [오류] 모달 순회 중 WebDriverException 발생. 재시도.")
        close_modal_if_open()
        human_scroll_cycle()
        open_first_modal()
        time.sleep(random.uniform(2.0, 4.0)) # 오류 후 긴 대기
        continue

# 3) 백업: 모달이 막히면 링크 방문 (게시 시간 저장 유지)
if len(results) < TARGET_CAPTIONS:
    close_modal_if_open()
    have = set()
    print("\n[백업 모드] 모달 순회 실패. 개별 URL 직접 방문으로 전환.")

    # URL 링크 수집 재시도 (느린 스크롤 적용)
    for i in range(40):
        anchors = driver.find_elements(By.CSS_SELECTOR, "a[href*='/p/'], a[href*='/reel/']")
        for a in anchors:
            href = a.get_attribute("href")
            if href and ("/p/" in href or "/reel/" in href):
                have.add(href)
        if len(have) >= TARGET_CAPTIONS * 1.5:
            break
        human_scroll_cycle()

    links = [h for h in have if h not in visited]
    for url in links:
        if len(results) >= TARGET_CAPTIONS: break

        # ---------------- 개별 방문 시 재시도 로직 (최대 2회) ----------------
        for attempt in range(2):
            try:
                crawl_iter += 1
                print(f"  ↳ [시도 #{attempt + 1}] 백업 방문: {url}")
                driver.get(url)

                # article 요소 로딩 명시적 대기
                wait.until(EC.presence_of_element_located((By.TAG_NAME, "article")))
                time.sleep(random.uniform(*VISIT_WAIT))

                url, cap, tags, post_time = extract_current() # 게시 시간 추출
                if url not in visited and (cap or tags):
                    visited.add(url)
                    results.append({"url": url, "caption": cap, "hashtags": tags, "posting_time": post_time})
                    print(f"  ↳ 수집 완료 ({len(results)}/{TARGET_CAPTIONS}): {url} (시간: {post_time[:19]})")
                    break
            except TimeoutException:
                print(f"  ↳ [오류] 타임아웃 발생 (재시도 필요): {url}")
                if attempt == 0: continue
                else: break
            except WebDriverException:
                print(f"  ↳ [오류] 웹 드라이버 예외 발생 (재시도 필요): {url}")
                if attempt == 0: continue
                else: break

# ---------------- CSV 저장 (헤더 포함) ----------------
with open(CSV_PATH, "w", newline="", encoding="utf-8-sig") as f:
    w = csv.writer(f)
    w.writerow(["url", "caption", "hashtags", "posting_time"]) # posting_time 헤더 유지
    for r in results:
        w.writerow([r["url"], r["caption"], ";".join(r["hashtags"]), r["posting_time"]])

print(f"\n[최종 완료] 캡션 {len(results)}건 저장 → {CSV_PATH}")
driver.quit()


[크롤링 #1] 모달 뷰에서 포스트 처리 중...
  ↳ 수집 완료 (1/300): https://www.instagram.com/p/DPJpak3CSxJ/liked_by/ (시간: 2025-09-28T15:20:40)
[크롤링 #2] 모달 뷰에서 포스트 처리 중...
  ↳ 수집 완료 (2/300): https://www.instagram.com/p/DQLcwdtCQQl/liked_by/ (시간: 2025-10-24T04:39:56)
[크롤링 #3] 모달 뷰에서 포스트 처리 중...
  ↳ 수집 완료 (3/300): https://www.instagram.com/p/DOmfnOyE5rW/liked_by/ (시간: 2025-09-14T23:41:35)
[크롤링 #4] 모달 뷰에서 포스트 처리 중...
  ↳ 수집 완료 (4/300): https://www.instagram.com/p/DQD97xXE-RN/liked_by/ (시간: 2025-10-21T06:57:26)
[크롤링 #5] 모달 뷰에서 포스트 처리 중...
  ↳ 수집 완료 (5/300): https://www.instagram.com/p/DPvSGOoElzK/liked_by/ (시간: 2025-10-13T06:13:13)
[크롤링 #6] 모달 뷰에서 포스트 처리 중...
  ↳ 수집 완료 (6/300): https://www.instagram.com/p/DQK_xYGE2Nq/liked_by/ (시간: 2025-10-24T00:26:39)
[크롤링 #7] 모달 뷰에서 포스트 처리 중...
  ↳ 수집 완료 (7/300): https://www.instagram.com/p/DOqAx4sEcOI/liked_by/ (시간: 2025-09-16T08:29:08)
[크롤링 #8] 모달 뷰에서 포스트 처리 중...
  ↳ 수집 완료 (8/300): https://www.instagram.com/p/DQJXENmCVHt/liked_by/ (시간: 2025-10-23T09:11:43)
[크롤링 #9] 모달 뷰에서 